# Telco Custumer Churn

Neste projeto irei construir um modelo de machine learnig para prever a possibilidade de um cliente cancelar o contrato com uma operadora de telecomunicações. Trata-se de um problema de aprendizado supervisionado (utilizando um conjunto de dados rotulado) de classificação, em que o alvo (target) é 1 se o cliente cancelou o serviço e 0 caso contrário.

Pipiline de resolução usado no projeto(baseado no CRISPIM-DM)
1. Definir o problema de negócio.

2. Coletar os dados e obter uma visão geral deles.

3. Dividir os dados em conjuntos de treino e teste.

4. Explorar os dados (Análise Exploratória de Dados – EDA).

5. Engenharia de atributos, limpeza e pré-processamento dos dados.

6. Treinamento dos modelos, comparação, seleção de atributos e ajuste de hiperparâmetros.

7. Teste final do modelo em produção e avaliação.

8. Conclusão e interpretação dos resultados do modelo.

9. Implantação (deploy).


Neste notebook, irei realizar a modelagem de machine learning, cobrindo as etapas 5 a 8 do pipeline apresentado acima. O principal objetivo aqui é construir um modelo que possa prever com precisão a probabilidade de um cliente cancelar (churn). Após construir esse modelo, a empresa de telecomunicações podera prever a probabilidade de um cliente sair, e direcionar suas estratégias para retenção e fidelização do cliente. Adquirir novos clientes é mais caro do que manter os existentes. Além disso, abordarei essas etapas em detalhes a seguir, explicando o motivo de cada decisão tomada.


In [1]:
# Visualização e manipulação do dataframe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.model_selection import train_test_split

## Carregando os dados 

In [2]:
data_patch = '../data/WA_Fn-UseC_-Telco-Customer-Churn.xls'

df = pd.read_csv(data_patch)

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.tail()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes
7042,3186-AJIEK,Male,0,No,No,66,Yes,No,Fiber optic,Yes,...,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.5,No


Irei transformar a coluna "TotalCharges" para o tipo float64, pois o mesmo é composta por dados númericos e substituir os valores nulos de "TotalCharges" por zero, pois os mesmo são referentes aos clientes que ainda não completaram um mês de serviço.

In [5]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [6]:
df.loc[df['tenure'] == 0, 'TotalCharges'] = 0

PROXIMOS PASSOS
1. Carregamento e Tipagem (Antes do Split)
mudar as variaveis objetos para cateory e mapear os valores



Irei remover algumas tabelas, cujos valores não contribuem de forma significativa para a análise, transformar as colunas que estão com objetos em catégorias e padronizar o nome das colunas (tenure e gender)

In [7]:
df = df.drop(columns=['customerID', 'gender'])

In [8]:
#padronizando nomes das colunas
df = df.rename(columns={ 'tenure' : 'Tenure' })

In [9]:
#modificando tipos das colunas 
categorical_columns = [
    'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn',
    'MultipleLines', 'InternetService', 'Contract', 'PaymentMethod', 
    'Gender', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'SeniorCitizen'
]

for col in categorical_columns:
    if col in df.columns:  
        df[col] = df[col].astype('category')

## 3. Separando os dados entre treino e test (Split)

* Irei separar os dados entre treino e teste para evitar o DataLekage, e garantir a capacidade de genereralização do modelo. 

In [14]:
X = df.drop(columns = ['Churn'])
y = df['Churn'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state =42)

print(f"Treino: {X_train.shape[0]} amostras | Teste: {X_test.shape[0]} amostras")

Treino: 5634 amostras | Teste: 1409 amostras


In [15]:
print(f"Treno: {y_train.value_counts(normalize = True)}")
print(f"Teste: {y_test.value_counts(normalize = True)}")

Treno: Churn
No     0.734647
Yes    0.265353
Name: proportion, dtype: float64
Teste: Churn
No     0.734564
Yes    0.265436
Name: proportion, dtype: float64


* as proporções entre 